<img src="images/cads-logo.png" style="height: 100px;padding-top:5px" align=left> <img src="images/apache_spark.png" style="height: 20%;width:20%; padding-top:0px" align=right>

MC Copy: https://colab.research.google.com/drive/1FIWKsUV0nwSuhbSvpifG6_Mwr9tms3-j?usp=sharing

# Manipulating Data using Apache Spark

In this notebook, we are going to get our hands dirty with Spark DataFrame API to perform common data operations.

In [8]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 63kB/s 
     |████████████████████████████████| 204kB 45.9MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=3c603b74c77938d4f9f1be4884458a169dc600033bf14e938ab7c4b23fb2db27
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [9]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

If you take a look at dataset folder, you will see `flights.csv` that contains a row for every flight that left Portland International Airport (PDX) or Seattle-Tacoma International Airport (SEA) in 2014 and 2015.

In the first step, we should create a DataFrame using `flights.csv` file and then create a table (temporaray view) for querying flights by using SQL commands. Let's do it.

In [ ]:
import os
MAIN_DIRECTORY = os.getcwd()
file_path =MAIN_DIRECTORY+"/dataset/flights.csv"

In [1]:
# Using this instead to access gdrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# cd to the current file path
cd "/content/drive/MyDrive/UM Lecture/CADS/13 BDA with Apache Spark 2day"

/content/drive/MyDrive/UM Lecture/CADS/13 BDA with Apache Spark 2day


In [13]:
df_flights = spark.read.format("csv").option("header","true").option('inferSchema','true').load("dataset/flights.csv")

In [ ]:
# A simple way to create a dataframe in Spark
# df_flights = spark.read.csv("dataset/flights.csv", header=True, inferSchema = True)

In [14]:
df_flights.createOrReplaceTempView('flights')

### Exercise 1: Use SQL to get the first five rows of the flights table and save the result to flights5, finally show the results. 

In [17]:
flights5 = spark.sql('SELECT * FROM flights LIMIT 5')

In [18]:
flights5.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
+----+-----+---+--------+---------+-----

### Exercise 2: Write a query that counts the number of flights to each airport from SEA and PDX.

In [29]:
spark.sql('SELECT count(*) FROM flights WHERE origin IN ("SEA" ,"PDX")').show()

+--------+
|count(1)|
+--------+
|   10000|
+--------+



In [108]:
# MC
spark.sql('SELECT origin, dest, count(*) FROM flights GROUP BY origin , dest').show()

+------+----+--------+
|origin|dest|count(1)|
+------+----+--------+
|   SEA| RNO|       8|
|   SEA| DTW|      98|
|   SEA| CLE|       2|
|   SEA| LAX|     450|
|   PDX| SEA|     144|
|   SEA| BLI|       5|
|   PDX| IAH|      57|
|   PDX| PHX|     209|
|   SEA| SLC|     225|
|   SEA| SBA|      23|
|   SEA| BWI|      29|
|   PDX| IAD|      23|
|   PDX| SFO|     305|
|   SEA| KOA|      40|
|   PDX| MCI|      15|
|   SEA| SJC|     213|
|   SEA| ABQ|      43|
|   SEA| SAT|      18|
|   PDX| ONT|      57|
|   SEA| LAS|     364|
+------+----+--------+
only showing top 20 rows



In [110]:
# MC 2nd way
df_flights.groupBy(['origin','dest']).count().show()

+------+----+-----+
|origin|dest|count|
+------+----+-----+
|   SEA| RNO|    8|
|   SEA| DTW|   98|
|   SEA| CLE|    2|
|   SEA| LAX|  450|
|   PDX| SEA|  144|
|   SEA| BLI|    5|
|   PDX| IAH|   57|
|   PDX| PHX|  209|
|   SEA| SLC|  225|
|   SEA| SBA|   23|
|   SEA| BWI|   29|
|   PDX| IAD|   23|
|   PDX| SFO|  305|
|   SEA| KOA|   40|
|   PDX| MCI|   15|
|   SEA| SJC|  213|
|   SEA| ABQ|   43|
|   SEA| SAT|   18|
|   PDX| ONT|   57|
|   SEA| LAS|  364|
+------+----+-----+
only showing top 20 rows



### Exercise 3: Write a piece of code to create a DataFrame using `airports.csv`, this file contains information about different airports all over the world. 

In [24]:
df_airports = spark.read.csv("dataset/airports.csv", header=True, inferSchema = True)

In [25]:
df_airports.show(5)

+---+--------------------+----------+-----------+----+---+---+
|faa|                name|       lat|        lon| alt| tz|dst|
+---+--------------------+----------+-----------+----+---+---+
|04G|   Lansdowne Airport|41.1304722|-80.6195833|1044| -5|  A|
|06A|Moton Field Munic...|32.4605722|-85.6800278| 264| -5|  A|
|06C| Schaumburg Regional|41.9893408|-88.1012428| 801| -6|  A|
|06N|     Randall Airport| 41.431912|-74.3915611| 523| -5|  A|
|09J|Jekyll Island Air...|31.0744722|-81.4277778|  11| -4|  A|
+---+--------------------+----------+-----------+----+---+---+
only showing top 5 rows



In [112]:
df_airports.printSchema()

root
 |-- faa: string (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- alt: integer (nullable = true)
 |-- tz: integer (nullable = true)
 |-- dst: string (nullable = true)



Let's look at performing column-wise operations. In Apache Spark, you can do this using the `.withColumn(colName, col)`  which returns a new DataFrame by adding a column or replacing the existing column that has the same name.

*Parameters*:  
- **colName** – string, name of the new column.
- **col** – a Column expression for the new column. 

The new `column` must be an object of class Column. Creating one of these is as easy as extracting a column from your DataFrame using `df.colName`.
Apache Spark DataFrame is **immutable**. Immutable means that it can't be changed, and so columns can't be updated in place.
For example:
```python
df = df.withColumn("newCol", df.oldCol + 1)
```
The above code creates a DataFrame with the same columns as df plus a new column, `newCol`, where every entry is equal to the corresponding entry from `oldCol`, plus one.

Sometimes we have to change a column data type to another one, in this case, we can use the following code:
```python
from pyspark.sql.functions import col
df_name = df_name.withColumn("columnName", col("columnName").cast("DataType"))
```

### Exercise 4: Update `flights` DataFrame to include a new column called `duration_hrs`, that contains the duration of each flight in hours.

In [52]:
from pyspark.sql.functions import col
df_flights = df_flights.withColumn("duration_hrs", df_flights['air_time'].cast("float")/60)

In [44]:
df_flights.show(5)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|      duration_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|               2.2|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|               6.0|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|              1.85|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|1.3833333333333333|
|2014|    3|  9|     754|  

In [114]:
# MC
from pyspark.sql.functions import col
df_flights = df_flights.withColumn("air_time", col('air_time').cast("integer"))
df_flights = df_flights.withColumn("duration_hrs", col('air_time')/60)

In [116]:
df_flights.show(5)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|      duration_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|   132.0|     954|   6|    58|               2.2|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|   360.0|    2677|  10|    40|               6.0|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|   111.0|     679|  14|    43|              1.85|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|    83.0|     569|  17|     5|1.3833333333333333|
|2014|    3|  9|     754|  

### Exercise 5: Write a query using the `.filter()` method to find all the flights that flew over 1000 miles.

In [48]:
df_flights.filter(df_flights.distance>1000).show(5)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-----------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|     duration_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-----------------+
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|              6.0|
|2014|    4| 19|    1236|       -4|    1508|       -7|     AS| N309AS|   490|   SEA| SAN|     135|    1050|  12|    36|             2.25|
|2014|   11| 19|    1812|       -3|    2352|       -4|     AS| N564AS|    26|   SEA| ORD|     198|    1721|  18|    12|              3.3|
|2014|    8|  3|    1120|        0|    1415|        2|     AS| N305AS|   656|   SEA| PHX|     154|    1107|  11|    20|2.566666666666667|
|2014|   11| 12|    2346|       -4

### Exercise 6: Write a query using `.filter()` method, to only keep flights from SEA to PDX. This query should only return `tailnum`, `origin`, and `dest` columns.

In [70]:
# Solution 1
df_flights.filter((df_flights['origin'] == 'SEA') & ( df_flights['dest'] == 'PDX')  ).select("tailnum", "origin", "dest").show(5)


+-------+------+----+
|tailnum|origin|dest|
+-------+------+----+
| N810SK|   SEA| PDX|
| N822SK|   SEA| PDX|
| N586SW|   SEA| PDX|
| N223SW|   SEA| PDX|
| N580SW|   SEA| PDX|
+-------+------+----+
only showing top 5 rows



In [117]:
#Solution 2
df_flights.filter((col('origin') == 'SEA') & ( col('dest') == 'PDX')  ).select("tailnum", "origin", "dest").show(5)

+-------+------+----+
|tailnum|origin|dest|
+-------+------+----+
| N810SK|   SEA| PDX|
| N822SK|   SEA| PDX|
| N586SW|   SEA| PDX|
| N223SW|   SEA| PDX|
| N580SW|   SEA| PDX|
+-------+------+----+
only showing top 5 rows



In [119]:
# MC solution 1
df_flights.filter(df_flights['origin'] == 'SEA').filter( df_flights['dest'] == 'PDX').select("tailnum", "origin", "dest").show(5)

+-------+------+----+
|tailnum|origin|dest|
+-------+------+----+
| N810SK|   SEA| PDX|
| N822SK|   SEA| PDX|
| N586SW|   SEA| PDX|
| N223SW|   SEA| PDX|
| N580SW|   SEA| PDX|
+-------+------+----+
only showing top 5 rows



In [120]:
# MC Solution 2
df_flights.filter((df_flights['origin'] == 'SEA') & ( df_flights['dest'] == 'PDX')).select("tailnum", "origin", "dest").show(5)


+-------+------+----+
|tailnum|origin|dest|
+-------+------+----+
| N810SK|   SEA| PDX|
| N822SK|   SEA| PDX|
| N586SW|   SEA| PDX|
| N223SW|   SEA| PDX|
| N580SW|   SEA| PDX|
+-------+------+----+
only showing top 5 rows



We can perform column-wise operations using `.select()` method. When we select a column using the `df.colName` notation. In `.select()` method, we can perform any column operation and it will return the transformed column. 
For example, the following command returns a column of flight durations in hours instead of minutes.
```python
df_flights.select(df_flights.air_time/60)
```
We can use the `alias()` method to rename a column we've selected. The following example shows how we can do that.
```python
df_flights.select((df_flights.air_time/60).alias("duration_hrs")
```
If we want to stick to the SQL syntax, we can use `.selectExpr()` method as well. The following commad is equivalent to the previous code.

```python
df_flights.selectExpr("air_time/60 as duration_hrs")
```

### Exercise 7: Write a query that return these columns, `origin`, `dest`, `tailnum`, and average speed in KM per hour.

In [77]:
#Solution 1
df_flights.select(df_flights.origin,df_flights.dest,df_flights.tailnum,(df_flights.distance/(df_flights.air_time/60)).alias("avg_speed")).show(5)

+------+----+-------+------------------+
|origin|dest|tailnum|         avg_speed|
+------+----+-------+------------------+
|   SEA| LAX| N846VA| 433.6363636363636|
|   SEA| HNL| N559AS| 446.1666666666667|
|   SEA| SFO| N847VA|367.02702702702703|
|   PDX| SJC| N360SW| 411.3253012048193|
|   SEA| BUR| N612AS| 442.6771653543307|
+------+----+-------+------------------+
only showing top 5 rows



In [81]:
#Solution 2
df_flights.selectExpr("origin", "dest", "tailnum", "distance/(air_time/60) as avg_speed").show(5)

+------+----+-------+------------------+
|origin|dest|tailnum|         avg_speed|
+------+----+-------+------------------+
|   SEA| LAX| N846VA| 433.6363636363636|
|   SEA| HNL| N559AS| 446.1666666666667|
|   SEA| SFO| N847VA|367.02702702702703|
|   PDX| SJC| N360SW| 411.3253012048193|
|   SEA| BUR| N612AS| 442.6771653543307|
+------+----+-------+------------------+
only showing top 5 rows



In [121]:
# MC solution 1
avg_speed = (df_flights.distance/df_flights.duration_hrs).alias('avg_speed')

In [122]:
type(avg_speed)

pyspark.sql.column.Column

In [123]:
df_flights.select('origin', 'dest', 'tailnum',avg_speed).show()

+------+----+-------+------------------+
|origin|dest|tailnum|         avg_speed|
+------+----+-------+------------------+
|   SEA| LAX| N846VA| 433.6363636363636|
|   SEA| HNL| N559AS| 446.1666666666667|
|   SEA| SFO| N847VA|367.02702702702703|
|   PDX| SJC| N360SW| 411.3253012048193|
|   SEA| BUR| N612AS| 442.6771653543307|
|   PDX| DEN| N646SW|491.40495867768595|
|   PDX| OAK| N422WN|             362.0|
|   SEA| SFO| N361VA| 415.7142857142857|
|   SEA| SAN| N309AS| 466.6666666666667|
|   SEA| ORD| N564AS| 521.5151515151515|
|   SEA| LAX| N323AS| 440.3076923076923|
|   SEA| PHX| N305AS|431.29870129870125|
|   SEA| LAS| N433AS| 409.6062992125984|
|   SEA| ANC| N765AS|474.75409836065575|
|   SEA| SFO| N713AS| 315.8139534883721|
|   PDX| SFO| N27205| 366.6666666666667|
|   SEA| SMF| N626AS|477.63157894736844|
|   SEA| MDW| N8634A|481.38888888888886|
|   SEA| BOS| N597AS| 516.4137931034483|
|   PDX| BUR| N215AG| 441.6216216216216|
+------+----+-------+------------------+
only showing top

In [124]:
# Solution 2
df_flights.selectExpr("origin", "dest", "tailnum", "distance/duration_hrs as avg_speed").show(5)

+------+----+-------+------------------+
|origin|dest|tailnum|         avg_speed|
+------+----+-------+------------------+
|   SEA| LAX| N846VA| 433.6363636363636|
|   SEA| HNL| N559AS| 446.1666666666667|
|   SEA| SFO| N847VA|367.02702702702703|
|   PDX| SJC| N360SW| 411.3253012048193|
|   SEA| BUR| N612AS| 442.6771653543307|
+------+----+-------+------------------+
only showing top 5 rows



### Exercise 8: Find the the shortest (in terms of distance) flight that left PDX by first filtering and using the `.min()` method. Perform the filtering by referencing the column directly, not passing a SQL string.

In [87]:
#Solution 1
df_flights.filter(df_flights["origin"]=="PDX").agg({"distance":"min"}).show()

+-------------+
|min(distance)|
+-------------+
|          106|
+-------------+



In [94]:
#Solutin 2
df_flights.groupBy("origin").min("distance").filter(df_flights["origin"]== "PDX").show()

+------+-------------+
|origin|min(distance)|
+------+-------------+
|   PDX|          106|
+------+-------------+



In [ ]:
# Solution 3


In [127]:
# MC Solution 1
df_flights.filter(df_flights.origin == 'PDX').groupBy('origin').agg({"distance":"min"}).show()

+------+-------------+
|origin|min(distance)|
+------+-------------+
|   PDX|          106|
+------+-------------+



In [128]:
# MC Solution 2
from pyspark.sql.functions import min
df_flights.filter(df_flights.origin == 'PDX').groupBy('origin').agg(min('distance')).show()

+------+-------------+
|origin|min(distance)|
+------+-------------+
|   PDX|          106|
+------+-------------+



In [129]:
# MC Solution 3
df_flights.filter(df_flights.origin == 'PDX').groupBy('origin').min('distance').show()

+------+-------------+
|origin|min(distance)|
+------+-------------+
|   PDX|          106|
+------+-------------+



### Exercise 9: Find the the longest (in terms of time) flight that left SEA by filtering and using the `.max()` method. Perform the filtering by referencing the column directly, not passing a SQL string.

if we run the following code, we will get an error, because `air_time` data type is string, first we should cast it to an integer column.

In [130]:
df_flights = df_flights.withColumn("air_time", df_flights['air_time'].cast("integer"))

In [131]:
df_flights.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- dep_time: string (nullable = true)
 |-- dep_delay: string (nullable = true)
 |-- arr_time: string (nullable = true)
 |-- arr_delay: string (nullable = true)
 |-- carrier: string (nullable = true)
 |-- tailnum: string (nullable = true)
 |-- flight: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- air_time: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- hour: string (nullable = true)
 |-- minute: string (nullable = true)
 |-- duration_hrs: double (nullable = true)



In [ ]:
#Solution 1

In [104]:
df_flights.filter(df_flights["origin"]=="SEA").agg({"air_time":"max"}).show()

+-------------+
|max(air_time)|
+-------------+
|        409.0|
+-------------+



In [ ]:
#Solution 2

In [106]:
df_flights.groupBy("origin").max("air_time").filter(df_flights["origin"]== "SEA").show()

+------+-------------+
|origin|max(air_time)|
+------+-------------+
|   SEA|        409.0|
+------+-------------+



In [132]:
# MC solution 1
from pyspark.sql.functions import *
df_flights.filter(df_flights["origin"]=="SEA").groupBy('origin').agg(max('air_time')).show()

+------+-------------+
|origin|max(air_time)|
+------+-------------+
|   SEA|          409|
+------+-------------+



In [134]:
# MC solution 2
df_flights.filter(df_flights["origin"]=="SEA").groupBy('origin').max("air_time").show()

+------+-------------+
|origin|max(air_time)|
+------+-------------+
|   SEA|          409|
+------+-------------+



### Exercise 10: Write a query that uses the `.avg()` method to get the average air time of Delta Airlines flights ( the carrier column value is "DL") that left SEA.

In [137]:
#MC
df_flights.filter((df_flights.carrier == "DL") & (df_flights.origin == "SEA")).groupBy("carrier").agg(round(avg("air_time"),2).alias('Avg. Flight Time')).show()

+-------+----------------+
|carrier|Avg. Flight Time|
+-------+----------------+
|     DL|          188.21|
+-------+----------------+



### Exercise 11: Write a query that uses the `.sum()` method to get the total number of hours all planes spent in the air by creating a column called `duration_hrs` from the column `air_time`.

In [138]:
df_flights.groupBy().sum('duration_hrs').show()

+------------------+
| sum(duration_hrs)|
+------------------+
|25289.600000000126|
+------------------+



### Exercise 12: Write a query that uses `tailnum` column to count the number of flights each plane made.

In [140]:
df_flights.groupBy('tailnum').count().orderBy('count',ascending = False).show()

+-------+-----+
|tailnum|count|
+-------+-----+
| N929EV|    1|
| N3JYAA|    1|
| N509JB|    1|
| N436AA|    1|
| N448UA|    1|
| N229WN|    1|
| N37274|    1|
| N8634A|    1|
| N633VA|    1|
| N958WN|    1|
| N961DN|    1|
| N8322X|    1|
| N240WN|    1|
| N567AA|    1|
|  N6700|    1|
| N466SW|    1|
| N4XUAA|    1|
| N927DN|    1|
| N251WN|    1|
| N336NW|    1|
+-------+-----+
only showing top 20 rows



### Exercise 13: Write a query that returns the average duration of flights from `PDX` and `SEA`.

In [141]:
df_flights.groupBy('origin').count().show()

+------+-----+
|origin|count|
+------+-----+
|   SEA| 6754|
|   PDX| 3246|
+------+-----+



In [142]:
df_flights.groupBy('origin').avg('air_time').show()

+------+------------------+
|origin|     avg(air_time)|
+------+------------------+
|   SEA| 160.4361496051259|
|   PDX|137.11543248288737|
+------+------------------+



### Exercise 14: Write a query that returns the average departure delay (`dep_delay`) in each month for each destination. Then import PySpark functions to calculate the standard deviation of `dep_delay` by using `stddev()` function.

In [144]:
df_flights = df_flights.withColumn("dep_delay",col("dep_delay").cast('integer'))

In [146]:
df_flights.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- dep_time: string (nullable = true)
 |-- dep_delay: integer (nullable = true)
 |-- arr_time: string (nullable = true)
 |-- arr_delay: string (nullable = true)
 |-- carrier: string (nullable = true)
 |-- tailnum: string (nullable = true)
 |-- flight: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- air_time: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- hour: string (nullable = true)
 |-- minute: string (nullable = true)
 |-- duration_hrs: double (nullable = true)



In [150]:
df_flights.groupBy('month','dest').avg('dep_delay').orderBy("month").show()

+-----+----+-------------------+
|month|dest|     avg(dep_delay)|
+-----+----+-------------------+
|    1| PDX|-5.6923076923076925|
|    1| LAS|  9.857142857142858|
|    1| DCA|               -1.8|
|    1| OAK|  8.290322580645162|
|    1| SBA|-0.6666666666666666|
|    1| JFK| 22.428571428571427|
|    1| PHL|               -1.0|
|    1| SLC|             6.5625|
|    1| MKE|                4.0|
|    1| DFW|  7.696969696969697|
|    1| JNU|                2.6|
|    1| ORD| 19.964285714285715|
|    1| SEA|  4.428571428571429|
|    1| GEG|  8.666666666666666|
|    1| KTN|                9.0|
|    1| RDM|             -1.625|
|    1| MSP|               11.2|
|    1| MCI|                0.5|
|    1| EWR| 13.076923076923077|
|    1| ABQ|               -3.0|
+-----+----+-------------------+
only showing top 20 rows



In [153]:
df_flights.groupBy('dest').agg(stddev('dep_delay'))).show()

SyntaxError: ignored

### Exercise 15: Write a query that performs left outer join on the flights and airports DataFrames.
- The flights and airports DataFrames are already in the workspace. 
- First, examine the airports DataFrame by calling .show() method. 
- Note which key column will let you join these two DataFrames.
- Before joining these two DataFrames, rename the `faa` column in `airports` to `dest`, and then convert this DataFrame to a temporary view (table).
- Use `spark.sql` to perform left outer join on these two tables. 


In [154]:
df_airports.show(5)

+---+--------------------+----------+-----------+----+---+---+
|faa|                name|       lat|        lon| alt| tz|dst|
+---+--------------------+----------+-----------+----+---+---+
|04G|   Lansdowne Airport|41.1304722|-80.6195833|1044| -5|  A|
|06A|Moton Field Munic...|32.4605722|-85.6800278| 264| -5|  A|
|06C| Schaumburg Regional|41.9893408|-88.1012428| 801| -6|  A|
|06N|     Randall Airport| 41.431912|-74.3915611| 523| -5|  A|
|09J|Jekyll Island Air...|31.0744722|-81.4277778|  11| -4|  A|
+---+--------------------+----------+-----------+----+---+---+
only showing top 5 rows



In [155]:
df_flights.show(5)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|      duration_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|               2.2|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|               6.0|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|              1.85|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|1.3833333333333333|
|2014|    3|  9|     754|  

In [156]:
df_airports = df_airports.withColumnRenamed('faa','dest')

In [157]:
df_airports.show(5)

+----+--------------------+----------+-----------+----+---+---+
|dest|                name|       lat|        lon| alt| tz|dst|
+----+--------------------+----------+-----------+----+---+---+
| 04G|   Lansdowne Airport|41.1304722|-80.6195833|1044| -5|  A|
| 06A|Moton Field Munic...|32.4605722|-85.6800278| 264| -5|  A|
| 06C| Schaumburg Regional|41.9893408|-88.1012428| 801| -6|  A|
| 06N|     Randall Airport| 41.431912|-74.3915611| 523| -5|  A|
| 09J|Jekyll Island Air...|31.0744722|-81.4277778|  11| -4|  A|
+----+--------------------+----------+-----------+----+---+---+
only showing top 5 rows



In [159]:
df_flights.createOrReplaceTempView('flights')
df_airports.createOrReplaceTempView('airports')

In [162]:
join_df = spark.sql("SELECT f.origin, f.dest , a.name, f.flight FROM flights as f \
                    LEFT JOIN airports as a ON f.dest = a.dest").show()

+------+----+--------------------+------+
|origin|dest|                name|flight|
+------+----+--------------------+------+
|   SEA| LAX|    Los Angeles Intl|  1780|
|   SEA| HNL|       Honolulu Intl|   851|
|   SEA| SFO|  San Francisco Intl|   755|
|   PDX| SJC|Norman Y Mineta S...|   344|
|   SEA| BUR|            Bob Hope|   522|
|   PDX| DEN|         Denver Intl|    48|
|   PDX| OAK|Metropolitan Oakl...|  1520|
|   SEA| SFO|  San Francisco Intl|   755|
|   SEA| SAN|      San Diego Intl|   490|
|   SEA| ORD|  Chicago Ohare Intl|    26|
|   SEA| LAX|    Los Angeles Intl|   448|
|   SEA| PHX|Phoenix Sky Harbo...|   656|
|   SEA| LAS|      Mc Carran Intl|   608|
|   SEA| ANC|Ted Stevens Ancho...|   121|
|   SEA| SFO|  San Francisco Intl|   306|
|   PDX| SFO|  San Francisco Intl|  1458|
|   SEA| SMF|     Sacramento Intl|   368|
|   SEA| MDW| Chicago Midway Intl|   827|
|   SEA| BOS|General Edward La...|    24|
|   PDX| BUR|            Bob Hope|  3488|
+------+----+--------------------+

### Exercise 16: Rewrite the previous query by using DataFrame API `.join` method. 

In PySpark, we can use `.join` method to perform joins. This method takes three arguments. 
- The first argument is the second DataFrame that we want to join with the first one. 
- The second argument, `on`, is the name of the key column(s) as a string. The names of the key column(s) must be the same in each table. 
- The third argument, `how`, specifies the kind of join to perform. 

To perform left outer join set the value of `how` to `"leftouter"`.

In [163]:
df_flights.join(df_airports,'dest','leftouter').select('origin',"dest","name","flight").show()

+------+----+--------------------+------+
|origin|dest|                name|flight|
+------+----+--------------------+------+
|   SEA| LAX|    Los Angeles Intl|  1780|
|   SEA| HNL|       Honolulu Intl|   851|
|   SEA| SFO|  San Francisco Intl|   755|
|   PDX| SJC|Norman Y Mineta S...|   344|
|   SEA| BUR|            Bob Hope|   522|
|   PDX| DEN|         Denver Intl|    48|
|   PDX| OAK|Metropolitan Oakl...|  1520|
|   SEA| SFO|  San Francisco Intl|   755|
|   SEA| SAN|      San Diego Intl|   490|
|   SEA| ORD|  Chicago Ohare Intl|    26|
|   SEA| LAX|    Los Angeles Intl|   448|
|   SEA| PHX|Phoenix Sky Harbo...|   656|
|   SEA| LAS|      Mc Carran Intl|   608|
|   SEA| ANC|Ted Stevens Ancho...|   121|
|   SEA| SFO|  San Francisco Intl|   306|
|   PDX| SFO|  San Francisco Intl|  1458|
|   SEA| SMF|     Sacramento Intl|   368|
|   SEA| MDW| Chicago Midway Intl|   827|
|   SEA| BOS|General Edward La...|    24|
|   PDX| BUR|            Bob Hope|  3488|
+------+----+--------------------+

#### Awesome